In [1]:
import pandas as pd

In [2]:
import os

In [ ]:
files = [f for f in os.listdir( if f.startswith('HC_2023_2024_ACT_DATA_HIST'))] #reading data about the acts
dfs = []
for file in files:
  data = pd.read_excel(file)
  dfs.append(data)
act_data = pd.concat(dfs, ignore_index=True)
act_data

In [ ]:
files = [f for f in os.listdir( if f.startswith('HC_2023_2024_ACT_DATA_HIST'))] #reading data about the acts
dfs = []
for file in files:
  data = pd.read_excel(file)
  dfs.append(data)
act_data = pd.concat(dfs, ignore_index=True)
act_data

In [ ]:
general = pd.read_excel('HC_2023_2024_GENERAL_DATA_HIST_20250211.xlsx')
general

In [ ]:
pogash = pd.read_excel('HC_2023_2024_TRAN_SUM_DATA_HIST_20250211.xlsx')
pogash

In [ ]:
files = [f for f in os.listdir( if f.startswith('CLID_CRM_DATA'))] #reading data about the acts
dfs = []
for file in files:
  data = pd.read_excel(file)
  dfs.append(data)
predictors = pd.concat(dfs, ignore_index=True)
predictors

In [ ]:
files = [f for f in os.listdir( if f.startswith('CLID_TRAN_DATA_PR'))] #reading data about the acts
dfs = []
for file in files:
  data = pd.read_excel(file)
  dfs.append(data)
predictors_2 = pd.concat(dfs, ignore_index=True)
predictors_2

In [ ]:
files = [f for f in os.listdir( if f.startswith('CLID_INFO_DATA'))] #reading data about the acts
dfs = []
for file in files:
  data = pd.read_excel(file)
  dfs.append(data)
predictors_3 = pd.concat(dfs, ignore_index=True)
predictors_3

Preparing model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import xgboost as xgb

In [ ]:
action_map = {
    "ДЗВОНОУ": 0,
    "МЕСЕНДЖЕР": 1,
    "ВИЕЗД": 2,
    "МЕСЕНДЖЕР 3-ОС": 3,
    "ПИСЬМО": 4,
    "ДЗВОНОК 3-ОС": 5,
    "ВИЕЗД 3-ОС": 6
}

act_data['action_encoded'] = act_data['action'].map(action_map)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [ ]:
model = xgb.XGBClassifier(
    objective='multi:softmax',
    num_class=7,
    n_estimators=100,
    max_depth=3,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42


In [ ]:
model.fit(X_train, y_train)


In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
inv_map = {v: k for k, v in action_map.items()}

predictions_text = [inv_map[i] for i in y_pred]
print(predictions_text[:20])


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report


param_grid = {
    'max_depth': [3, 5, 7],
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.1, 0.05, 0.01],
    'gamma': [0, 0.1, 0.3],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'reg_alpha': [0, 0.5, 1],
    'reg_lambda': [1, 2]
}

xgb_clf = xgb.XGBClassifier(
    objective='multi:softmax',
    num_class=7,
    random_state=42
)

grid = GridSearchCV(
    estimator=xgb_clf,
    param_grid=param_grid,
    scoring='accuracy',
    cv=3,
    verbose=2,
    n_jobs=-1
)

grid.fit(X_train, y_train)
best_model = grid.best_estimator_

print("Best params from GridSearch:")
print(grid.best_params_)

y_pred_best = best_model.predict(X_test)
print("\nClassification Report on Test Set:")
print(classification_report(y_test, y_pred_best))


In [ ]:
xgb.plot_importance(model)

In [ ]:
xgb.plot_importance(best_model)